In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ========== CONFIG ==========
data_dir = "/content/drive/MyDrive/Images"  # This is the folder with 'lung' and 'non-lung' subfolders
batch_size = 16
num_epochs = 10
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# ========== TRANSFORMS ==========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]) if device.type == "cpu" else transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
# ========== DATASET & DATALOADER ==========
dataset = datasets.ImageFolder(data_dir, transform=transform)
class_names = dataset.classes  # Will be ['lung', 'non-lung'] or similar

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

In [ ]:
# ========== LOAD ViT ==========
vit = models.vit_b_16(weights=None)  # or vit_b_16(weights="IMAGENET1K_V1") if using pretrained
vit.heads.head = nn.Linear(vit.heads.head.in_features, 2)
vit = vit.to(device)

# ========== TRAINING SETUP ==========
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit.parameters(), lr=learning_rate)

# ========== TRAIN LOOP ==========
best_acc = 0.0
for epoch in range(num_epochs):
    vit.train()
    total, correct = 0, 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)
        outputs = vit(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    print(f"Train Accuracy: {train_acc:.4f}")


Epoch 1/10 - Training: 100%|██████████| 5/5 [03:24<00:00, 40.86s/it]


Train Accuracy: 0.6875


Epoch 2/10 - Training: 100%|██████████| 5/5 [02:55<00:00, 35.05s/it]


Train Accuracy: 0.9250


Epoch 3/10 - Training: 100%|██████████| 5/5 [02:51<00:00, 34.34s/it]


...

Train Accuracy: 0.9625


Epoch 9/10 - Training: 100%|██████████| 5/5 [02:54<00:00, 34.90s/it]


Train Accuracy: 0.9500


Epoch 10/10 - Training: 100%|██████████| 5/5 [02:52<00:00, 34.45s/it]

Train Accuracy: 0.9625


In [ ]:
# ====== VALIDATION ======
vit.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Validating"):
        images = images.to(device)
        outputs = vit(images)
        preds = torch.argmax(outputs, dim=1).cpu()
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

val_acc = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {val_acc:.4f}")

if val_acc > best_acc:
    best_acc = val_acc
    torch.save(vit.state_dict(), "vit_lung_classifier.pth")
    print("✅ Best model saved.")

print("\nTraining complete! Best Validation Accuracy:", best_acc)


Validating: 100%|██████████| 2/2 [00:19<00:00,  9.96s/it]


Validation Accuracy: 1.0000
✅ Best model saved.

Training complete! Best Validation Accuracy: 1.0
